
---

# 🗃️ FastAPI CRUD – No Pydantic (using SQLite)

A simple FastAPI app that performs basic CRUD operations using SQLite, without using Pydantic models.

---

## 🚀 How to Run

```bash
uvicorn crud:app --reload
```

---

## ⚙️ Database Setup

### 🧩 Imports & App (top of file)

```python
from contextlib import asynccontextmanager
import sqlite3
from typing import Dict, Any, List, Optional

from fastapi import FastAPI, HTTPException, Body, status
```

```python
app = FastAPI(title="FastAPI CRUD (No Pydantic)")
DB_PATH = "employee.db"
```

### 🔧 Connect to SQLite

```python
def get_db() -> sqlite3.Connection:
    conn = sqlite3.connect(DB_PATH, check_same_thread=False)
    conn.row_factory = sqlite3.Row
    return conn
```

### 🏗️ Create Table

Auto-creates the table if not already present.

```python
def init_db() -> None:
    with get_db() as db:
        db.execute("""
            CREATE TABLE IF NOT EXISTS employee (
                id   INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT    NOT NULL,
                age  INTEGER NOT NULL,
                city TEXT    NOT NULL
            )
        """)
        db.commit()
```

```python
@asynccontextmanager
async def lifespan(_: FastAPI):
    init_db()   # ensure table exists at startup
    yield

app.router.lifespan_context = lifespan
```

---

## 📄 Endpoints Summary

| Method | Endpoint                         | Description                   |
| ------ | -------------------------------- | ----------------------------- |
| GET    | `/`                              | Home / landing page           |
| POST   | `/create`                        | Create new employee record    |
| GET    | `/employee/view`                 | View all employee records     |
| GET    | `/employee/view/{employee_id}`   | View a single employee record |
| PUT    | `/employee/update/{employee_id}` | Update an existing employee   |
| DELETE | `/employee/delete/{employee_id}` | Delete an employee            |

---

## 🔍 API Endpoints – With Code

### 🧰 (Small helpers used by endpoints)

```python
def row_to_dict(r: sqlite3.Row) -> Dict[str, Any]:
    return {"id": r["id"], "name": r["name"], "age": r["age"], "city": r["city"]}

def get_one(db: sqlite3.Connection, employee_id: int) -> Optional[sqlite3.Row]:
    cur = db.execute("SELECT id, name, age, city FROM employee WHERE id = ?", (employee_id,))
    return cur.fetchone()

def ensure_exists(db: sqlite3.Connection, employee_id: int) -> None:
    if get_one(db, employee_id) is None:
        raise HTTPException(status_code=404, detail="Employee not found")
```

---

### 🏠 1. Home Page

```python
@app.get("/")
def home_page():
    return {"home": "Welcome to landing page of FastAPI"}
```

---

### ➕ 2. Create Employee

> Accepts **JSON body** with `name`, `age`, `city` (kept simple, no Pydantic).

```python
@app.post("/create", status_code=status.HTTP_201_CREATED)
def insert_record(payload: Dict[str, Any] = Body(...)):
    name = (payload.get("name") or "").strip()
    age  = payload.get("age")
    city = (payload.get("city") or "").strip()

    if not name or not isinstance(age, int) or age < 0 or not city:
        raise HTTPException(status_code=422, detail="Invalid payload. Expect {name:str, age:int>=0, city:str}")

    with get_db() as db:
        cur = db.execute(
            "INSERT INTO employee (name, age, city) VALUES (?, ?, ?)",
            (name, age, city)
        )
        db.commit()
        new_id = cur.lastrowid
        row = get_one(db, new_id)
        return {"message": "employee record inserted successfully", "employee": row_to_dict(row)}
```

---

### 📋 3. View All Employees

```python
@app.get("/employee/view")
def view_records():
    with get_db() as db:
        cur = db.execute("SELECT id, name, age, city FROM employee ORDER BY id ASC")
        employees = cur.fetchall()
    return [row_to_dict(emp) for emp in employees]
```

---

### 👁️ 4. View Single Employee by ID

```python
@app.get("/employee/view/{employee_id}")
def single_view_record(employee_id: int):
    with get_db() as db:
        employee = get_one(db, employee_id)
        if employee is None:
            raise HTTPException(status_code=404, detail="Employee not found")
    return row_to_dict(employee)
```

---

### 🔄 5. Update Employee

> Full update; expects **JSON body** with all fields: `name`, `age`, `city`.

```python
@app.put("/employee/update/{employee_id}")
def single_update_record(employee_id: int, payload: Dict[str, Any] = Body(...)):
    name = (payload.get("name") or "").strip()
    age  = payload.get("age")
    city = (payload.get("city") or "").strip()

    if not name or not isinstance(age, int) or age < 0 or not city:
        raise HTTPException(status_code=422, detail="Invalid payload. Expect {name:str, age:int>=0, city:str}")

    with get_db() as db:
        ensure_exists(db, employee_id)
        db.execute(
            "UPDATE employee SET name = ?, age = ?, city = ? WHERE id = ?",
            (name, age, city, employee_id)
        )
        db.commit()
        row = get_one(db, employee_id)
        return {
            "message": f"Employee id {employee_id} record updated successfully",
            "employee": row_to_dict(row)
        }
```

---

### ❌ 6. Delete Employee

```python
@app.delete("/employee/delete/{employee_id}")
def single_record_delete(employee_id: int):
    with get_db() as db:
        ensure_exists(db, employee_id)
        db.execute("DELETE FROM employee WHERE id = ?", (employee_id,))
        db.commit()
    return {"message": f"Employee id {employee_id} record deleted successfully"}
```

---
